## A.1 Import libraries

In [1]:
import pandas as pd
import matplotlib.pyplot as plt 
import numpy as np 
import statsmodels.api as sm
import re

## A.2 Import data

### A.2.1 Import dataframe

In [2]:
df = pd.read_stata('COVID19_Mexico_13.04.2020_version1.dta')
df.head(2)

,FECHA_ACTUALIZACION,ENTIDAD_UM,sexo,ENTIDAD_NAC,ENTIDAD_RES,MUNICIPIO_RES,TIPO_PACIENTE,FECHA_INGRESO,FECHA_SINTOMAS,FECHA_DEF,...,cardiovascular,obesidad,RENAL_CRONICA,tabaquismo,OTRO_CASO,resultado,migrante,PAIS_NACIONALIDAD,PAIS_ORIGEN,uci
0,2020-04-13,15,1,15,15,37,2,2020-04-09,2020-03-28,9999-99-99,...,2,2,2,2,2,1,99,99,97,1
1,2020-04-13,28,2,16,28,32,1,2020-04-06,2020-04-04,9999-99-99,...,2,2,2,2,1,1,99,99,97,97


## A.3 Map variables

### A.3.1 Import categorical variables description
#### A.3.1.1 Show all sheet names

In [3]:
df = pd.read_stata('COVID19_Mexico_13.04.2020_version1.dta')

xls = pd.ExcelFile("Catalogos_English.xlsx")
sheets = xls.sheet_names
sheets = sheets
sheets

['Catálogo SEXO',
 'Catálogo TIPO_PACIENTE',
 'Catálogo NACIONALIDAD',
 'Catálogo RESULTADO',
 'Catálogo de ENTIDADES',
 'Catálogo MUNICIPIOS']

#### A.3.1.2 Lower case sheet names without catalogo

In [4]:
col_names = [i.split(' ', 1)[1] for i in sheets]
col_names = [i.lower() for i in col_names]
col_names

['sexo',
 'tipo_paciente',
 'nacionalidad',
 'resultado',
 'de entidades',
 'municipios']

#### A.3.1.3 Match sheet names to existing df columns

In [5]:
matching_cols = []
for col in col_names:
    res = df.filter(regex=re.compile(str(col)[:-1], re.I), axis=1).columns.to_list()
    matching_cols.append(res)
matching_cols = [x for x in matching_cols if x != []] 
matching_cols

[['sexo'],
 ['TIPO_PACIENTE'],
 ['nacionalidad', 'PAIS_NACIONALIDAD'],
 ['resultado'],
 ['MUNICIPIO_RES']]

##### A.3.1.3.1 list of list $\rightarrow$ one list

In [6]:
flatten = lambda l: [item for sublist in l for item in sublist]
matching_cols = flatten(matching_cols)
matching_cols

['sexo',
 'TIPO_PACIENTE',
 'nacionalidad',
 'PAIS_NACIONALIDAD',
 'resultado',
 'MUNICIPIO_RES']

### A.3.2 Maping

#### A.3.2.1 Data is originally in following format

In [7]:
pd.read_excel("Catalogos_English.xlsx", sheet_name=0, index_col=1)

,CLAVE
DESCRIPCIÓN,
FEMALE,1
MALE,2
NOT SPECIFIED,99


### A.3.2.2 Mapping the data into new columns

In [8]:
for (sheet,col) in zip(sheets, matching_cols):
    df_sheet = pd.read_excel("Catalogos_English.xlsx", sheet_name=sheet, index_col=1)
    dicts = df_sheet.iloc[:,0].to_dict()  
    dicts = {dicts[k] : k for k in dicts}
    df[str(col)+"_new"] = df[col].replace(dicts)
df.iloc[:, -6:].head(3)

,sexo_new,TIPO_PACIENTE_new,nacionalidad_new,PAIS_NACIONALIDAD_new,resultado_new,MUNICIPIO_RES_new
0,FEMALE,HOSPITALIZIED,MEXICAN,99,AGUASCALIENTES,HUHÍ
1,MALE,OUT PATIENT,MEXICAN,99,AGUASCALIENTES,ESPITA
2,MALE,HOSPITALIZIED,MEXICAN,99,BAJA CALIFORNIA,MONTE ESCOBEDO


## A.4. Import english column names data

In [9]:
col_names_df = pd.read_excel('Descriptores_English.xlsx', usecols=[1,2])
col_names_df.head(3)

,NOMBRE DE VARIABLE,DESCRIPCIÓN DE VARIABLE
0,FECHA_ACTUALIZACION,Date of database elaboration
1,ENTIDAD_UM,State where medical attention was received
2,SEXO,Gender of patient


### A.4.1 Replace names in df with english names from "col_names_df"

In [10]:
new_col_names = col_names_df.iloc[:,1].to_list() #New column name into list
old_df = df.iloc[:, :len(new_col_names)] #Take old columns
old_df.columns = new_col_names #Rename columns
df = pd.concat([old_df,df.filter(regex="new",axis=1)], axis=1) #Combine the new named columns 
df.head(3)

,Date of database elaboration,State where medical attention was received,Gender of patient,State of birth of patient,State of residence of patient,Municipality of residence of patient,Type of patient,Date entering medical facility,Date of symptoms,Date of death,...,Patient is migrant,Nationality of patient,Which country is patient from,Patient entered ICU,sexo_new,TIPO_PACIENTE_new,nacionalidad_new,PAIS_NACIONALIDAD_new,resultado_new,MUNICIPIO_RES_new
0,2020-04-13,15,1,15,15,37,2,2020-04-09,2020-03-28,9999-99-99,...,99,99,97,1,FEMALE,HOSPITALIZIED,MEXICAN,99,AGUASCALIENTES,HUHÍ
1,2020-04-13,28,2,16,28,32,1,2020-04-06,2020-04-04,9999-99-99,...,99,99,97,97,MALE,OUT PATIENT,MEXICAN,99,AGUASCALIENTES,ESPITA
2,2020-04-13,15,2,15,15,31,2,2020-04-06,2020-04-04,9999-99-99,...,99,99,97,2,MALE,HOSPITALIZIED,MEXICAN,99,BAJA CALIFORNIA,MONTE ESCOBEDO


# Questions

## 1.1 How many individuals have been tested for covid19? 

In [11]:
ans_1_1 = len(df.loc[:,'Laboratory results of covid19 test'])
print('{} people have been tested for Covid-19'.format(ans_1_1))

38670 people have been tested for Covid-19


## 1.2 What proportion have tested positive? 

### 1.2.1 Create confirmed dummy

In [12]:
df['confirmed_dummy'] = df['Laboratory results of covid19 test'].replace({2:0})

### 1.2.2 Drop the obs that are awaiting test results

In [13]:
df =  df.loc[df['Laboratory results of covid19 test']!=3,:]

### 1.2.3 Answer: What proportion have tested positive

In [14]:
ans_1_2 = df.loc[:, 'confirmed_dummy'].mean() 
print('{:.2%} people tested positive'.format(ans_1_2))

17.10% people tested positive


## 1.3 By gender

In [15]:
ans_1_3 = df.groupby('sexo_new')['confirmed_dummy'].mean()
ans_1_3.to_frame().rename(columns={"Laboratory results of covid19 test": "Postive rate"})

,confirmed_dummy
sexo_new,
FEMALE,0.137714
MALE,0.206960


## 2. What proportion of individuals testing positive have passed away? 

## 2.1 Find death column

In [16]:
death_colname = df.filter(regex='death').columns.to_list()
death_colname

['Date of death']

## 2.2 Create death dummy

In [17]:
df['death_dummy'] = list(map(lambda x: 0 if x == '9999-99-99' else 1, df['Date of death']))

## 2.3 Answer: What proportion of individuals testing positive have passed away? 

In [18]:
ans_2_3 = float(df[['death_dummy']].mean())
print('{:.2%} tested positive have died'.format(ans_2_3))

2.11% tested positive have died


## 2.4 Answer: Passed away by gender

In [19]:
round(df.groupby('sexo_new')['death_dummy'].mean()*100,2).astype('str')+'%'

sexo_new
FEMALE    1.46%
MALE      2.83%
Name: death_dummy, dtype: object

# 3. Analyze the probability of passing away by pre existing conditions and age/gender using a regression analysis. 

## 3.1 Regression by age and gender

In [20]:
df['female_dummy'] = df['Gender of patient'].replace({2:0})

In [21]:
y = df.death_dummy
x = df[["female_dummy", 'Age of patient']]
x = sm.add_constant(x)
sm.OLS(y,x).fit().summary()

/Users/corybaird/opt/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2542: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            death_dummy   R-squared:                       0.024
Model:                            OLS   Adj. R-squared:                  0.024
Method:                 Least Squares   F-statistic:                     358.1
Date:                Sun, 03 May 2020   Prob (F-statistic):          2.18e-154
Time:                        10:22:13   Log-Likelihood:                 15606.
No. Observations:               29329   AIC:                        -3.121e+04
Df Residuals:                   29326   BIC:                        -3.118e+04
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==================================================================================
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
const             -0.0204      0.002     -9.035      0.000      -0.025      -0.016
female_dummy      -0.0128      0.002     -7.703      0.000      -0.016      -0.010
Age of patient     0.0012   4.72e-05     25.459      0.000       0.001       0.001
==============================================================================
Omnibus:                    33973.224   Durbin-Watson:                   1.964
Prob(Omnibus):                  0.000   Jarque-Bera (JB):          2191194.427
Skew:                           6.421   Prob(JB):                         0.00
Kurtosis:                      43.350   Cond. No.                         127.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

## 3.2 Regression by age gender and pre-existing conditions

### 3.2.1 Create dummy variables for pre-existing condition variables

In [22]:
for col in ['Patient has asthma', 'Patient has immonsupression','Patient has hypertension', 'Patient has other illness','Patient has cardiovascular illnesses', 'Patient is obese','Patient has chronic insufficient renal syndrome']:
    df[str(col)] = df[str(col)].replace({2:0})

### 3.2.2 Regression

In [23]:
y = df.death_dummy
x = df[["female_dummy", 'Age of patient', 'Patient has asthma', 'Patient has immonsupression','Patient has hypertension', 'Patient has other illness','Patient has cardiovascular illnesses', 'Patient is obese','Patient has chronic insufficient renal syndrome']]
x = sm.add_constant(x)
sm.OLS(y,x).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            death_dummy   R-squared:                       0.026
Model:                            OLS   Adj. R-squared:                  0.026
Method:                 Least Squares   F-statistic:                     87.07
Date:                Sun, 03 May 2020   Prob (F-statistic):          1.09e-160
Time:                        10:22:13   Log-Likelihood:                 15639.
No. Observations:               29329   AIC:                        -3.126e+04
Df Residuals:                   29319   BIC:                        -3.118e+04
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
===================================================================================================================
                                                      coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------------------
const                                              -0.0203      0.002     -8.981      0.000      -0.025      -0.016
female_dummy                                       -0.0128      0.002     -7.693      0.000      -0.016      -0.010
Age of patient                                      0.0012   4.73e-05     25.295      0.000       0.001       0.001
Patient has asthma                                 -0.0016      0.001     -2.367      0.018      -0.003      -0.000
Patient has immonsupression                         0.0034      0.000      7.119      0.000       0.002       0.004
Patient has hypertension                            0.0008      0.000      1.647      0.100      -0.000       0.002
Patient has other illness                          -0.0006      0.000     -1.838      0.066      -0.001     3.9e-05
Patient has cardiovascular illnesses               -0.0003      0.001     -0.433      0.665      -0.002       0.001
Patient is obese                                   -0.0010      0.000     -2.115      0.034      -0.002   -7.23e-05
Patient has chronic insufficient renal syndrome    -0.0002      0.001     -0.240      0.810      -0.001       0.001
==============================================================================
Omnibus:                    33923.901   Durbin-Watson:                   1.964
Prob(Omnibus):                  0.000   Jarque-Bera (JB):          2182424.791
Skew:                           6.405   Prob(JB):                         0.00
Kurtosis:                      43.271   Cond. No.                         128.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""